In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from rdkit import Chem

import xgboost as xgboost
import catboost as catboost
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from rdkit.Chem import AllChem

In [2]:
cuma_branch_sol_emi = pd.read_csv("../data/cuma_branch in D4C DB/cuma_branch_sol_emi.csv")

# combination of substituents(position1, position5)

In [ ]:
# connsidering position1 and position5
locom_int_list=[[1,5]]

# 20 common subgstituents smiles
common_20_subgroup_df = pd.read_csv('../data/common_20_subgroups.csv')
common_20_subgroup=np.array(common_20_subgroup_df['sub smiles'])

# Union of all substituents at each position in DB and 20 common substituents"
a=list(cuma_branch_sol_emi['1'].drop_duplicates())
a= list(set(a) | set(common_20_subgroup))

e=list(cuma_branch_sol_emi['5'].drop_duplicates())
e= list(set(e) | set(common_20_subgroup))

subgroup_6_smiles=[]
sub_list=[a, np.array([]), np.array([]),np.array([]),e,np.array([])]

for li in sub_list:
    subgroup_6_smiles.append(li)

In [4]:
# all combination of substituent smiles
from tqdm import tqdm
from itertools import product

chunk_size = 10000

input_smiles_final = []
current_batch = []


for locom in locom_int_list:
    subsmiles_in_comb = [subgroup_6_smiles[locations-1] for locations in locom]
    for combo in product(*subsmiles_in_comb):
        branch_stand = ['loc1','loc2','loc3','loc4','loc5','loc6']
        for idx, loc in enumerate(locom):
            branch_stand[loc-1] = combo[idx]
        current_batch.append(branch_stand)     
        if len(current_batch) >= chunk_size:
            input_smiles_final.append(np.vstack(current_batch))
            current_batch =[]


if current_batch:
    input_smiles_final.append(np.vstack(current_batch))
    
input_smiles_final_array = np.vstack(input_smiles_final)

# Using XGBoost model for emission wavelength

In [ ]:
from functions.conv_input_Gauss_function import one_touch_representation_Gauss

cuma_branch_sol_emi = pd.read_csv("../data/cuma_branch in D4C DB/cuma_branch_sol_emi.csv")
X_emi= one_touch_representation_Gauss(cuma_branch_sol_emi,10,  10*( 0.6645613384222082/10), True)

Y_emi = np.array(cuma_branch_sol_emi['Emi']).flatten()
scaler = StandardScaler()
scaled_Y_emi = scaler.fit_transform(Y_emi.reshape(-1,1)).flatten()


xgb = xgboost.XGBRegressor(n_estimators = 89, learning_rate= 0.12170962342182974, subsample =0.6765177031554993, max_depth = 9, random_state=42)
xgb.fit(X_emi, scaled_Y_emi)

# augment & predict (in DMF)

In [6]:
def augmentes_X_conv_input_in_sol(solvent, sub_smiles_combo, conv_range, sigma_ratio):

    
    chunk_size = 10000
    current_batch = []
    augmented_x_conv=[]
    
    
    #solvent_conv
    sol_feature, sol_adj = _make_matrix([solvent], conv_range, sigma_ratio, True) 
    sol_matirx = np.dot(sol_adj[0], sol_feature[0])
    sol_conv_1D = np.dot(np.ones(sol_adj[0].shape[0]),sol_matirx)
    
    
    cuma_mol = Chem.MolFromSmiles('O=C1C=CC2=CC=CC=C2O1')
    core_sub_site_order = [2,3,5,6,7,8]
    
    #만든 조합 중에 폐기 되는 애들 인덱스 수집
    deleted_combi=[]
    for n in tqdm(range(len(sub_smiles_combo))):
        
        # get_sub_atom_index
        core = [Chem.MolFromSmarts('[#8]=[#6:1]1:[#6:2](-[#101:3]):[#6:4](-[#102:5]):[#6:6]2:[#6:7](-[#103:8]):[#6:9](-[#104:10]):[#6:11](-[#105:12]):[#6:13](-[#106:14]):[#6:15]:2:[#8:16]:1')]
    
        where_sub_is=[]
        what_sub_is=[]
        skip_to_next_n = False 
        
        for sub_loc, sub_smiles in enumerate(sub_smiles_combo[n]):
            if sub_smiles[0]=='*':

                chain = Chem.MolFromSmarts(sub_smiles.replace('*','',1))
                if chain is None:
                    print(f"Skipping invalid SMILES: {sub_smiles}, moving to next n")
                    deleted_combi.append(n)
                    skip_to_next_n = True 
                    break 
                
                core = Chem.ReplaceSubstructs(core[0], Chem.MolFromSmarts(f'[#{sub_loc+101}]'),chain)
                if core is None or not isinstance(core, tuple):
                    print(f"Invalid core after ReplaceSubstructs for sub_loc {sub_loc}, sub_smiles {sub_smiles}")
                    deleted_combi.append(n)
                    skip_to_next_n = True 
                    break 
                where_sub_is.append(sub_loc)
                what_sub_is.append(sub_smiles)
                
            else : 
                core = Chem.ReplaceSubstructs(core[0], Chem.MolFromSmarts(f'[#{sub_loc+101}]'),Chem.MolFromSmarts('[#1]'))
                continue
    
        if skip_to_next_n:
            continue  

        try:
            whole_mol = Chem.MolFromSmiles(Chem.MolToSmiles(core[0]))
        except Exception as e:
            print(f"Error in MolToSmiles : smiles which is augmented: {e}")
            deleted_combi.append(n)
            continue
        
        core_idx = whole_mol.GetSubstructMatches(cuma_mol)[0]
    
        sub_atomidx_list = []
        for m in range(len(where_sub_is)):
            sub_idx_candidates = whole_mol.GetSubstructMatches(Chem.MolFromSmiles(what_sub_is[m].replace('*','C',1)))
            sub_idx_candidates = np.array(sub_idx_candidates)
    
            core_site = np.isin(sub_idx_candidates, core_idx[core_sub_site_order[where_sub_is[m]]])
            scam_check = np.isin(sub_idx_candidates, core_idx)
            pick_correct_one = np.where((core_site.any(axis=1))&(np.sum(scam_check, axis=1)==1))[0][0]
            
            sub_idx = list(sub_idx_candidates[pick_correct_one])
            sub_idx.remove( core_idx[core_sub_site_order[where_sub_is[m]]] )
            sub_atomidx_list.append(sub_idx)
    
        
        # making Gaussian weighted graph convolution matirx
        new_feature, new_adj = _make_matrix([Chem.MolToSmiles(core[0])], conv_range, sigma_ratio, True)
        feature_matirx = np.dot(new_adj[0], new_feature[0])
    
    
        #making subgruop_modular_GGC_input
        all_subs_conv_1D = np.zeros(6*45)
        for x in range(len(sub_atomidx_list)):
            sub_conv = feature_matirx[sub_atomidx_list[x],:]
            sub_conv_1D = np.dot(np.ones(len(sub_atomidx_list[x])), sub_conv)
            all_subs_conv_1D[where_sub_is[x]*45:(where_sub_is[x]+1)*45] = sub_conv_1D
        if sub_atomidx_list:   
            del_sub_atoms = np.hstack(sub_atomidx_list)
        else:
            del_sub_atoms = np.array([])

        if del_sub_atoms.size > 0 :
            core_conv = np.delete(feature_matirx, del_sub_atoms, axis=0)
        else:
            core_conv = feature_matirx.copy()

        core_conv_1D = np.dot(np.ones(feature_matirx.shape[0]-len(del_sub_atoms)), core_conv)
    
        conv_branch = np.hstack((sol_conv_1D, core_conv_1D, all_subs_conv_1D)) 
        
        current_batch.append(conv_branch)
        if len(current_batch) >= chunk_size:
                augmented_x_conv.append(np.vstack(current_batch))
                current_batch =[]
    
    

    if current_batch:
        augmented_x_conv.append(np.vstack(current_batch))
        
    augmented_x_conv_final = np.vstack(augmented_x_conv)
    return augmented_x_conv_final, deleted_combi

In [ ]:
from functions.conv_input_Gauss_function import _make_matrix, predict_chunky
import gc

for sol, solname in zip(['CN(C)C=O'], ['DMF']):
    print(f'solvent : {solname} ({sol})')
    augmented_x_conv_final, deleted_combi = augmentes_X_conv_input_in_sol(sol, input_smiles_final_array, 10, 0.6645613384222082)

    all_predict = predict_chunky(augmented_x_conv_final, 10000, xgb)
    all_predict = scaler.inverse_transform(all_predict.reshape(-1,1)).reshape(all_predict.shape[0],)
    np.save(f'augment_predict_{solname}', all_predict)

    del all_predict  
    gc.collect()  

# Attribution of 20 substituents to emission wavelength (in DMF)

In [8]:
solname = 'DMF'
predict_on_sol = np.load(f"augment_predict_{solname}.npy")

In [ ]:
input_smiles_final_array=np.delete(input_smiles_final_array, deleted_combi, axis=0)

location_chunk = 20
delta_basic_20=[]
delta_basic_6loc=[]

for loc in [0,4]:
    for subidx in np.arange(20):
        
        predict_stand = predict_on_sol[input_smiles_final_array[:,loc] == common_20_subgroup[subidx]] # in DMF
        print(f'loc : {loc}, index : {subidx}',len(predict_stand))
    
        sub_combo_observe = input_smiles_final_array[input_smiles_final_array[:,loc] == common_20_subgroup[subidx]].astype(object).copy()
        sub_combo_observe[:,loc] = 'deleted'
        input_compare, deletes_combi2 = augmentes_X_conv_input_in_sol('CN(C)C=O', sub_combo_observe, 10, 0.6645613384222082) 
        
        predict_compare = predict_chunky(input_compare, 10000, xgb)
        predict_compare = scaler.inverse_transform(predict_compare.reshape(-1,1)).reshape(predict_compare.shape[0],)
        delta = predict_stand - predict_compare
        delta_basic_20.append(delta)
    
        if len(delta_basic_20) >= location_chunk:
            delta_basic_6loc.append(delta_basic_20)
            delta_basic_20 = []

In [ ]:
common_subgroup_shift_DMF = pd.DataFrame([])
for idx,loc in enumerate([1,5]):
    common_subgroup_shift_DMF[f'location {loc}'] = [np.mean(delta_basic_6loc[idx][n]) for n in range(20)]
common_subgroup_shift_DMF.index=common_20_subgroup_df['sub name']
common_subgroup_shift_DMF.index.name = 'functional group'
print(common_subgroup_shift_DMF)

In [12]:
common_subgroup_shift_DMF.to_csv('common_subgroup_shift_DMF_[1,5].csv',index = False)

In [ ]:
import seaborn as sns

for idx, position in enumerate([1,5]) :
    fig = plt.figure(figsize=(10,7))
    violin = sns.violinplot(data=delta_basic_6loc[idx])
    plt.title(f'Attribution of functional groups (position{position})', fontsize =27)
    plt.xlim(-1,19.5)
    violin.set_xticks(range(20))
    violin.set_xticklabels(list(common_20_subgroup_df['sub name']), rotation =90, fontsize =25)
    plt.ylabel('Emission λ shift (nm)', fontsize =25)
    plt.yticks(fontsize =22)
    sns.lineplot(x=np.arange(-1,21), y=0, linestyle='--', color ='gray')
   
    plt.tight_layout() 
    fig = violin.get_figure()
    plt.show()